In [1]:
import copy
import numpy as np
import numpy.linalg as la
import time
from collections import deque
import sys

In [ ]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.nodes = []
        self.width = None
        self.height = None
        self.end = [] 
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        self.expandedNumber = 0
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char == "%":
                    self.maze_list.append(0)
                elif char == ".":
#                     self.maze_list.append("G")
                    self.maze_list.append(1)
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char == "P":
#                     self.maze_list.append("P")
                    self.maze_list.append(1)
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))

    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]                
    
#   Transform the index in the raw_list to the index in the maze_list
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
        
    def getCordinate(self, point):
        return (point%self.width, point//self.width)
    
    
#   Return the adjacant node who is unvisited and valid
    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent
    
    
    def bfs(self):
        self.expandedNumber = 0
        self.bfsHelper()
       
    def bfsHelper(self):
        self.visited = []
        
        queue = []
        parent = {}
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
        self.expandedNumber += 1
        while queue:
            current = queue[0]
            queue = queue[1:]
            
            if current in self.end:
                print("I get to the destination!")
                # The last step to the solution, track back
                self.solution.append(current)

                while current != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution = self.solution[::-1]
                
                return True
                
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                self.expandedNumber += 1
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))     
        
        return True
    
    def manhattanDistance(self, cur, des): return abs(cur[0] - des[0]) + abs(cur[1] - des[1])

    
#   Create MST; Find the shortest path;
    def dijsktra(self, start):
#       Record the path; key: node   value: pre;
        shortestPathTree = {start: (-1,-1)}
#       Distance from the node to the starting point; key: node   value: distance;
        disToStart = {start:0}
        
#       Initialize the distance record; ESP, starting point has distance 0 and other nodes has maxINT;
        for node in range(self.width*self.height):
            if self.getCordinate(node) != start and self.maze_list[node] != 0:
                disToStart[self.getCordinate(node)] = sys.maxsize
#       self.nodes saves all the nodes; Check __init__;
        nodes = self.nodes[:]
#       Travel all the nodes, label their distance and pre;
        while nodes:
            min_node = None
#           Find the visited node with least distance;
            for node in nodes:
#             for node in shortestPathTree:
                if node in shortestPathTree:
                    if min_node == None:
                        min_node = node
                    elif disToStart[node] < disToStart[min_node]:
                        min_node = node
#           All the nodes have already been checked;
            if min_node == None:
                break
            
            nodes.remove(min_node)
#           Current cost from start to the node;
            minToStart = disToStart[min_node]
            
            adjacent_nodes = self.adjacentList(min_node)
#           Label all the adjacent nodes;
            for adj_node in adjacent_nodes:
#               The edge weight is only 1;
#                 newToStart = minToStart + 1
                newToStart = minToStart + self.manhattanDistance(min_node, self.end)
#               If the node has not been visited or the distance should be updated;
                if adj_node not in shortestPathTree or newToStart < disToStart[adj_node]:
                    shortestPathTree[adj_node] = min_node
                    disToStart[adj_node] = newToStart
#       Draw the solution;
        end = self.end
        solution = self.raw_list[:] # faster than deep copy;  
        while end != (-1,-1):
            x, y = end
            if self.raw_list[y][x] == ".":
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif self.raw_list[y][x] == "P":
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
            end = shortestPathTree[end]
        return solution
    
    def drawSolution(self):
        solution = self.raw_list[:] # faster than deep copy;  
        for pos in self.solution:
            x, y = pos
            if self.raw_list[y][x] == ".":
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif self.raw_list[y][x] == "P":
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
        return solution